In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline

# Both stabilize and measure in same plot

In [ ]:
from analysis.lib.single_click_ent import Phase_stab_and_meas as psm; reload(psm)
psm.analyze_phase(contains = 'SweepThetaXX',lt3_analysis = True, do_only_meas = False, do_only_stable = False, do_both = True)

# Only stabilize in plot

In [ ]:
from analysis.lib.single_click_ent import Phase_stab_and_meas as psm; reload(psm)
psm.analyze_phase(contains = 'SweepThetaXX',lt3_analysis = True, do_only_meas = False, do_only_stable = True, do_both = False)

# Only measure in plot

In [ ]:
from analysis.lib.single_click_ent import Phase_stab_and_meas as psm; reload(psm)
psm.analyze_phase(contains = 'SweepThetaXX',lt3_analysis = True, do_only_meas = True, do_only_stable = False, do_both = False)